### Entanglement-based Quantum Key Distribution in a $n$ x $n$ grid topology using Netsquid

#### The objetive of this project is to use the Ekert Protocol in order to implement QKD in a $n$ x $n$ Quantum Network. The following list describes the milestones to be achieved:
* Entangle a random route using Entanglement Swapping (Completed)
* Entangle two or more disjunctive routes (Completed)
* Generate the shared key in each node (In process)  

#### Code functionality: This implementation creates a $n$ x $n$ grid Quantum Network. After the Quantum Network is defined, the simulation starts dynamic protocols depending on the connection requirements. When the simulation ends, it brings back the path, fidelity and time information.

_refs_: 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater_chain.html 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater.html 
* Evan Sutcliffe, Matty J. Hoban & Alejandra Beghelli - Multipath Routing for Multipartite State Distribution in Quantum Networks  

_(adapted by D-Cryp7 for Netsquid 1.1.6)_

#### Limitations:
* Conjunctive routes causes a Race Condition error. For now, our aim focus on disjunctive routes, due to the possible alternative options.

In [1]:
# Library imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
sys.path.insert(0, '../')
from custom_netsquid_functions import *
from base_netsquid_functions import *
from util import *

In [4]:
def setup_datacollector(network, protocol, path, node_qconnections):
    """
    Setup the datacollector to calculate the fidelity
    when the CorrectionProtocol has finished.

    Parameters
    ----------
    network : :class:`~netsquid.nodes.network.Network`
        Repeater chain network to put protocols on.

    protocol : :class:`~netsquid.protocols.protocol.Protocol`
        Protocol holding all subprotocols used in the network.

    Returns
    -------
    :class:`~netsquid.util.datacollector.DataCollector`
        Datacollector recording fidelity data.

    """
    def random_basis():
        r = randint(0, 1)
        return ns.Z if r else ns.X

    def measure(q, obs):
        measurement_result, prob = ns.qubits.measure(q, obs)
        if measurement_result == 0:
            state = "|0>" if obs == ns.Z else "|+>"
        else:
            state = "|1>" if obs == ns.Z else "|->"
        return ("Z" if obs == ns.Z else "X"), measurement_result, prob

    # Ensure nodes are ordered in the chain:
    # nodes = [network.nodes[name] for name in sorted(network.nodes.keys())]
    nodes = [network.nodes[str(name)] for name in path]
    A_port = int(node_qconnections[nodes[0].name][nodes[1].name].split("-")[0][-1])
    B_port = int(node_qconnections[nodes[-1].name][nodes[-2].name].split("-")[0][-1])
    nodes[0].shared_secret, nodes[-1].shared_secret = [], []
    
    nodes[0].entangled_qubits, nodes[-1].entangled_qubits = {}, {}
    entangled = False
    entangle_attemps = 0
    def calc_fidelity(evexpr):
        nonlocal entangled, entangle_attemps
        qubit_a, = nodes[0].qmemory.peek([A_port])
        qubit_b, = nodes[-1].qmemory.peek([B_port])
        fidelity = ns.qubits.fidelity([qubit_a, qubit_b], ks.b00, squared = True)
        measure_a = measure(qubit_a, random_basis())
        measure_b = measure(qubit_b, random_basis())
        if fidelity >= 0.9:
            if not entangled:
                print(f"{path} entangled in {entangle_attemps} attemps: storing qubits")
                nodes[0].entangled_qubits[f"{nodes[-1].name}"] = qubit_a
                nodes[-1].entangled_qubits[f"{nodes[0].name}"] = qubit_b
                entangled = True
            # QKD part
            if measure_a[0] == measure_b[0]:
                nodes[0].shared_secret.append(str(int(measure_a[1])))
                nodes[-1].shared_secret.append(str(int(measure_b[1])))
        else:
            entangle_attemps += 1
        return {"fidelity": fidelity, 
                "A_measure": measure_a[0], "A_measure_result": measure_a[1], 
                "B_measure": measure_b[0], "B_measure_result": measure_b[1]}

    dc = DataCollector(calc_fidelity, include_entity_name=False)
    dc.collect_on(pydynaa.EventExpression(source=protocol.subprotocols['CorrectProtocol'],
                                          event_type=Signals.SUCCESS.value))
    return dc

#### Entangle-Retry method

In [5]:
%%time

import time

# Default values

n = 3
node_distance = 20
num_iters = 1024
est_runtime = (0.5 + 2 * n - 1) * node_distance * 5e3


ns.sim_reset()
network, node_qconnections = network_setup(n, node_distance = node_distance,
                        source_frequency = 1e9 / est_runtime)

qdf = pandas.DataFrame(node_qconnections)
qdf = qdf.fillna("")

path = get_random_route(network)
while len(path) <= 3:
    path = get_random_route(network)
paths = []
for i in range(len(path) - 1):
    paths.append( [path[i], path[i + 1]] )

# print(paths)

traffic = {
    "path": paths
}

print(f"Simulation from paths: {paths}")

df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)

Simulation from paths: [[(2, 0), (1, 0)], [(1, 0), (1, 1)], [(1, 1), (0, 1)], [(0, 1), (0, 2)]]
[(1, 0), (1, 1)] entangled in 1 attemps: storing qubits
[(2, 0), (1, 0)] entangled in 1 attemps: storing qubits
[(1, 1), (0, 1)] entangled in 2 attemps: storing qubits
[(0, 1), (0, 2)] entangled in 6 attemps: storing qubits
CPU times: user 7.23 s, sys: 323 ms, total: 7.55 s
Wall time: 7.55 s


In [9]:
network.nodes[str(paths[0][0])].entangled_qubits

{'(1, 0)': Qubit('qsource_qconn_(1, 0)<->(2, 0)-#2-1')}

In [10]:
len(network.nodes[str(paths[0][0])].shared_secret)

246

In [11]:
len(network.nodes[str(paths[0][1])].shared_secret) # OJOOOOO

488